In [1]:
import datetime as dt
import numpy as np
import pandas as pd
from itertools import combinations

from processor.dart import DART_FUNDAMENTAL_PROCESSOR
from processor.fdr import FDR_INFO_PROCESSOR, FDR_OHLCV_PREPROCESSOR

from loader.pykrx_loader import get_pykrx_loader

from processor.factor.fundamental import FUNDAMENTAL_FACTOR_PROCESSOR
from processor.factor.technical import MOVING_AVERAGE_FACTOR_PROCESSOR
from processor.factor.trader import TRADER_FACTOR_PROCESSOR
from processor.factor.parameter import FACTOR_PARAMETER_PROCESSOR
from processor.factor.filter import FACTOR_FILTER

from analyser.factor import FACTOR_ANALYSER

In [2]:
CFG = {
    "analysis_date": "2023-11-01",
    "FUNDAMENTAL": {
        "FACTOR_PROCESSOR_CFG": {
            "ohlcv_recent_n": 10,
        },
        "FACTOR_ANALYSIS_CFG": {
            "profit_recent_n": 20,
        },
    },
    "MOVING_AVERAGE": {
        "FACTOR_PROCESSOR_CFG": {
            "factors": ["Close", "VolumeRotation"],
            "short_term_window": round((7 * 2) * (5 / 7)),
            "long_term_window": round((7 * 8) * (5 / 7)),
            "signal_recent_n": 20,
        },
        "FACTOR_ANALYSIS_CFG": {
            "profit_recent_n": 20,
        },
    },
    "TRADER": {
        "FACTOR_PROCESSOR_CFG": {
            "start": (dt.date.today() - dt.timedelta(days=7)).strftime("%Y-%m-%d"),
            "end": (dt.date.today() - dt.timedelta(days=1)).strftime("%Y-%m-%d"),
            # "end": dt.date.today().strftime("%Y-%m-%d"),
        },
        "FACTOR_FILTER_CFG": {
            "absolute_conditions": [
                "Corp > 0",
                "Foreign > 0",
                "Indivisual < 0",
            ]
        },
    },
}

In [3]:
# dart_fundamental
dart_fundamental_df = pd.read_csv("./data/dart_fundamental_df.csv", index_col=0)
dart_fundamental_processor = DART_FUNDAMENTAL_PROCESSOR(dart_fundamental_df)

In [4]:
# fdr_info
fdr_info_df = pd.read_csv("./data/fdr_info_df.csv", index_col=0)
fdr_info_processor = FDR_INFO_PROCESSOR(fdr_info_df)

In [5]:
# fdr_ohlcv
fdr_ohlcv_df = pd.read_csv("./data/fdr_ohlcv_df.csv", index_col=0)
fdr_ohlcv_preprocessor = FDR_OHLCV_PREPROCESSOR(fdr_ohlcv_df)
pps_fdr_ohlcv_df = fdr_ohlcv_preprocessor.get_pps_fdr_ohlcv_df()

In [6]:
# split ohlcv
future_ohlcv_df = pps_fdr_ohlcv_df[CFG["analysis_date"] < pps_fdr_ohlcv_df["Date"]].copy()
params_ohlcv_df = pps_fdr_ohlcv_df[pps_fdr_ohlcv_df["Date"] < CFG["analysis_date"]].copy()

In [7]:
# Fundamental Parameter

In [8]:
# get fundamental_factors_df
fundamental_factor_processor = FUNDAMENTAL_FACTOR_PROCESSOR(
    params_ohlcv_df, dart_fundamental_processor, fdr_info_processor
)
fundamental_factors_df = fundamental_factor_processor.get_fundamental_factor_df(
    CFG["FUNDAMENTAL"]["FACTOR_PROCESSOR_CFG"]
)

In [9]:
# Analysis
factor_analyser = FACTOR_ANALYSER(fundamental_factors_df)

In [10]:
factors = [col for col in fundamental_factors_df.columns if col != "StockCode"]
factor_combs = list(combinations(factors, 2))

params_dict = dict()

for factor_comb in factor_combs:
    profit_analysis_2d_df = factor_analyser.get_profit_analysis_2d_df(
        future_ohlcv_df, CFG["FUNDAMENTAL"]["FACTOR_ANALYSIS_CFG"], factor_comb
    )
    profit_analysis_2d_df.fillna(profit_analysis_2d_df.mean().mean(), inplace=True)

    factor_parameter_processor = FACTOR_PARAMETER_PROCESSOR(profit_analysis_2d_df, 3, 3)
    best_args = factor_parameter_processor.get_best_args()
    best_value = factor_parameter_processor.get_best_value()
    params_dict[best_args] = best_value

In [11]:
params_df = pd.DataFrame().from_dict(params_dict, orient="index",columns=['Value'])

In [12]:
params_df.nlargest(5, "Value")

,Value
"((TEPP, 0.0, 0.3), (CAPP, 0.7, 1.0))",0.128517
"((TAPP, 0.7, 1.0), (TEPP, 0.2, 0.5))",0.107244
"((TAPP, 0.6, 0.9), (CAPP, 0.0, 0.3))",0.092093
"((CLR, 0.2, 0.5), (TLR, 0.6, 0.9))",0.089866
"((TLR, 0.0, 0.3), (CEPP, 0.3, 0.6))",0.088302


In [13]:
fudamental_parameters = list(params_df.nlargest(5, "Value").index)
CFG["FUNDAMENTAL"]["FACTOR_FILTER_CFG"] = {
    "quantile_conditions": list(fudamental_parameters[0]),
}

In [14]:
# Technical parameter

In [15]:
# get factors_df
moving_average_factor_processor = MOVING_AVERAGE_FACTOR_PROCESSOR(params_ohlcv_df, fdr_info_processor)
moving_average_factors_df = moving_average_factor_processor.get_moving_average_factor_df(
    CFG['MOVING_AVERAGE']['FACTOR_PROCESSOR_CFG']
)

In [16]:
# Analysis
factor_analyser = FACTOR_ANALYSER(moving_average_factors_df)

In [17]:
factors = [col for col in moving_average_factors_df.columns if col != "StockCode"]
factor_combs = list(combinations(factors, 2))

In [18]:
params_dict = dict()

for factor_comb in factor_combs:
    profit_analysis_2d_df = factor_analyser.get_profit_analysis_2d_df(
        future_ohlcv_df, CFG['MOVING_AVERAGE']['FACTOR_ANALYSIS_CFG'], factor_comb
    )
    profit_analysis_2d_df.fillna(profit_analysis_2d_df.mean().mean(), inplace=True)

    factor_parameter_processor = FACTOR_PARAMETER_PROCESSOR(profit_analysis_2d_df, 3, 3)
    best_args = factor_parameter_processor.get_best_args()
    best_value = factor_parameter_processor.get_best_value()
    params_dict[best_args] = best_value

In [19]:
params_df = pd.DataFrame().from_dict(params_dict, orient="index",columns=['Value'])

In [20]:
params_df.nlargest(5, "Value")

,Value
"((Close, 0.6, 0.9), (VolumeRotation, 0.0, 0.3))",0.096154


In [21]:
technical_parameters = list(params_df.nlargest(5, "Value").index)
CFG["MOVING_AVERAGE"]["FACTOR_FILTER_CFG"] = {
    "quantile_conditions": list(technical_parameters[0]),
}

In [22]:
# Model

In [23]:
ohlcv_df = pps_fdr_ohlcv_df.copy()

In [24]:
# Fundamental
fundamental_factor_processor = FUNDAMENTAL_FACTOR_PROCESSOR(ohlcv_df, dart_fundamental_processor, fdr_info_processor)
fundamental_factor_df = fundamental_factor_processor.get_fundamental_factor_df(
    CFG["FUNDAMENTAL"]["FACTOR_PROCESSOR_CFG"]
)

factor_filter = FACTOR_FILTER(fundamental_factor_df)
fundamental_stockcodes = factor_filter.get_filtered_stockcodes(CFG["FUNDAMENTAL"]["FACTOR_FILTER_CFG"])

In [25]:
# Technical
## Moving Average
moving_average_factor_processor = MOVING_AVERAGE_FACTOR_PROCESSOR(ohlcv_df, fdr_info_processor)
moving_average_factors_df = moving_average_factor_processor.get_moving_average_factor_df(
    CFG["MOVING_AVERAGE"]["FACTOR_PROCESSOR_CFG"]
)

factor_filter = FACTOR_FILTER(moving_average_factors_df)
technical_stockcodes = factor_filter.get_filtered_stockcodes(
    CFG["MOVING_AVERAGE"]["FACTOR_FILTER_CFG"], fundamental_stockcodes
)

In [26]:
len(technical_stockcodes)

7

In [27]:
# Trader
pykrx_loader = get_pykrx_loader()

trader_factor_processor = TRADER_FACTOR_PROCESSOR(pykrx_loader)
trader_factor_df = trader_factor_processor.get_trader_factor_df(
    technical_stockcodes, CFG["TRADER"]["FACTOR_PROCESSOR_CFG"]
)
factor_filter = FACTOR_FILTER(trader_factor_df)
final_stockcodes = factor_filter.get_filtered_stockcodes(CFG["TRADER"]["FACTOR_FILTER_CFG"], technical_stockcodes)

In [28]:
final_stockcodes

{'000320', '016880'}